In [1]:
from selenium import webdriver
import time
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from PIL import Image  
import PIL
import math
import urllib
from urllib.request import urlretrieve
import cv2
import os
import numpy as np
import pandas as pd


service = Service(executable_path='./chromedriver.exe')
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)



In [111]:
urls = []
urls_seller = []
ids_item = []
ids_seller = []
shippings = []
names = []
categorys = []
prices = []
imgs = []

In [112]:
list_link = ['https://vi.aliexpress.com/w/wholesale-pants-for-men.html?page=', '&g=y&SearchText=pants+for+men',
             'https://vi.aliexpress.com/w/wholesale-sweaters-for-men.html?page=', '&g=y&SearchText=sweaters+for+men',
             'https://vi.aliexpress.com/w/wholesale-blazer-suits-for-men.html?page=', '&g=y&SearchText=blazer+suits+for+men',
             'https://vi.aliexpress.com/w/wholesale-Fashion-Suits.html?isFromCategory=y&categoryUrlParams=%7B%22q%22%3A%22Fashion+Suits%22%2C%22s%22%3A%22qp_nw%22%2C%22osf%22%3A%22categoryNagivateOld%22%2C%22sg_search_params%22%3A%22on___%2528%2520prism_tag_id%253A%25271000343408%2527%2520%2529%22%2C%22guide_trace%22%3A%2282afa9a3-78c5-41b5-bded-82e6bab7891f%22%2C%22scene_id%22%3A%2230630%22%2C%22searchBizScene%22%3A%22openSearch%22%2C%22recog_lang%22%3A%22en%22%2C%22bizScene%22%3A%22categoryNagivateOld%22%2C%22guideModule%22%3A%22unknown%22%2C%22postCatIds%22%3A%22200000343%2C200001813%22%2C%22scene%22%3A%22category_navigate%22%7D&page=', '&g=y&SearchText=Fashion+Suits',
             'https://vi.aliexpress.com/w/wholesale-down-jacket-men.html?page=', '&g=y&SearchText=down+jacket+men',
             'https://vi.aliexpress.com/w/wholesale-jeans-for-men.html?page=', '&g=y&SearchText=jeans+for+men',
             'https://vi.aliexpress.com/w/wholesale-shorts-for-men.html?page=', '&g=y&SearchText=shorts+for+men',
             'https://vi.aliexpress.com/w/wholesale-jackets-for-men.html?page=', '&g=y&SearchText=jackets+for+men',
             'https://vi.aliexpress.com/w/wholesale-shirts-for-men.html?page=', '&g=y&SearchText=shirts+for+men',
             'https://vi.aliexpress.com/w/wholesale-shoes-for-men.html?page=', '&g=y&SearchText=shoes+for+men',
             'https://vi.aliexpress.com/w/wholesale-shoes-for-Lady.html?page=', '&g=y&SearchText=shoes+for+Lady',]

In [114]:
def cao_trang(str_link1, str_link2, n_page = 20):
    # Thực hiện cuộn xuống cuối trang bằng JavaScript
    page = n_page+1
    for i in range(1, page, 1):
        url_category_page = str_link1 + str(i) + str_link2
        driver.get(url_category_page)

        category = driver.find_element(By.CSS_SELECTOR, '.pc-header--search--3hnHLKw .search--search--C_fO_MU .search--keyword--15P08Ji').get_attribute('value')

        time.sleep(2)

        driver.execute_script("window.scrollTo(0, document.body.scrollHeight-4000);")

        time.sleep(2)

        driver.execute_script("window.scrollTo(0, document.body.scrollHeight-2700);")

        time.sleep(2)

        driver.execute_script("window.scrollTo(0, document.body.scrollHeight-1000);")
        time.sleep(7)  # Nghỉ 30s để tránh bị chặn API 

        #tên của item
        name = driver.find_elements(By.CSS_SELECTOR, '.list--galleryWrapper--29HRJT4 .search-card-item .multi--title--G7dOCj3')


        #người bán
        owner = driver.find_elements(By.CSS_SELECTOR, '.list--galleryWrapper--29HRJT4 .cards--storeLink--XkKUQFS')

        price = driver.find_elements(By.CSS_SELECTOR, '.multi--price--1okBCly')


        #link của item
        links = driver.find_elements(By.CSS_SELECTOR, ".list--galleryWrapper--29HRJT4 .search-card-item")  # Vị trí chứa đường dẫn sản phẩm
        n = len(links)
        for k in range(n):
            # lấy id của sản phẩm
            link = links[k].get_attribute("href")
            urls.append(link)
            id_item = link[31:52]
            ids_item.append(id_item)

            # lấy id của người bán
            seller = owner[k].get_attribute("href")
            urls_seller.append(seller)
            id_seller = urls_seller[k][32:]
            ids_seller.append(id_seller)

            # lấy tên sản phẩm 
            names.append(name[k].get_attribute("title"))

            #lấy category
            categorys.append(category)

            #lấy giá
            prices.append(price[k].text)


In [116]:
for i in range(0, 60, 2):
    cao_trang(list_link[i], list_link[i+1], n_page = 5)

In [118]:
data = {'Id_item': ids_item, 'Id_seller': ids_seller, 'Name_item': names, 'Price': prices, 'Category': categorys}
df = pd.DataFrame(data) 
df.to_csv('item_new1.csv', index=False, encoding='utf-8')  

In [120]:
doc = pd.read_csv('item_new1.csv')
print(doc.shape)
doc[23542:23544]

(7875, 5)


,Id_item,Id_seller,Name_item,Price,Category


In [85]:
frames = [doc, doc1]
result = pd.concat(frames)
print(result.shape)
result[23542:23547]

(45882, 5)


,Id_item,Id_seller,Name_item,Price,Category
23542,32856605620.html,1103102470,It's Yiiya Strapless Asymmetrical Elegant Luxu...,"₫1,029,584",special occasion dresses for women luxury
23543,1005004789890418,912178471,Beyprern Beautiful Sleevless Cut-Out Crystal P...,"₫581,627",special occasion dresses for women luxury
0,1005005614397913,1102773884,2023 New Women'S Suit Jacket Spring Autumn Wom...,"₫371,082",curve plus size clothes for women
1,32909832407.html,1190823,Plus Size 5XL 150kg Spring Sweet Dress Women B...,"₫482,717",curve plus size clothes for women
2,4000983249894.ht,1190823,Plus Size 8XL 150kg Women Summer Modal Long Dr...,"₫385,883",curve plus size clothes for women


In [88]:
result.drop_duplicates(subset=['Id_item'], inplace=True)
result.to_csv('result.csv', index=False, encoding='utf-8')  
print(result.shape)

(43046, 5)


In [6]:
n = len(ids_item)
path = 'image'
#Nếu như có sẵn thư mục image rồi thì bỏ qua dòng lệnh tạo thư mục
try:
    os.makedirs(path)
except:
    pass
os.chdir(path)
for i in range(5):
    time.sleep(5)
    url_item = 'https://vi.aliexpress.com/item/' + ids_item[i] + '.html'
    driver.get(url_item)
    
    img = driver.find_element(By.XPATH,'//*[@id="root"]/div/div[3]/div/div[1]/div[1]/div[1]/div/div/div[2]/div[1]/div/div[1]/img')
    path_url = img.get_attribute("src")
    img = ids_item[i] + '.jpg'
    urlretrieve(path_url, img)
    
    color = driver.find_elements(By.XPATH, '//div[@class="sku-item--image--mXsHo3h"]/img')
    num_color = len(color)
    color_texts = color[0].get_attribute('alt')
    for num in range(1, num_color, 1):
        color_text = color[num].get_attribute('alt')
        color_texts = color_texts + ', ' + color_text
    num_colors.append(num_color)
    color_texts_list.append(color_texts)
    
    driver.execute_script("arguments[0].click();", WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, '//*[@class="comet-v2-btn comet-v2-btn-slim comet-v2-btn-large specification--btn--CXRSSZD comet-v2-btn-important"]'))))
    time.sleep(2)
    specifications = driver.find_elements(By.XPATH, '//div[@class="specification--prop--RejitI8"]')
    num_spe = len(specifications)
    
    spe_text = specifications[0].text.replace('\n', ': ')
    for num in range(1, num_spe, 1):
        spe = specifications[num].text.replace('\n', ': ')
        spe_text = spe_text + ', ' + spe
    spe_texts.append(spe_text)

    try:
        core = driver.find_element(By.XPATH, '//div[@class="reviewer--wrap--sPGWrNq"]/strong').text
    except:
        core = math.nan
    cores.append(core)
    
    owner_member_id = driver.find_element(By.XPATH, '//*[@id="root"]/div/div[1]/div/div[2]/a')
    omi = owner_member_id.get_attribute('href')[-10:]
    owner_member_id_list.append(omi)
os.chdir(r'C:\Users\ACER\Downloads\uyen_crawl')

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="root"]/div/div[1]/div/div[2]/a"}
  (Session info: chrome=119.0.6045.200); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6B5B482B2+55298]
	(No symbol) [0x00007FF6B5AB5E02]
	(No symbol) [0x00007FF6B59705AB]
	(No symbol) [0x00007FF6B59B175C]
	(No symbol) [0x00007FF6B59B18DC]
	(No symbol) [0x00007FF6B59ECBC7]
	(No symbol) [0x00007FF6B59D20EF]
	(No symbol) [0x00007FF6B59EAAA4]
	(No symbol) [0x00007FF6B59D1E83]
	(No symbol) [0x00007FF6B59A670A]
	(No symbol) [0x00007FF6B59A7964]
	GetHandleVerifier [0x00007FF6B5EC0AAB+3694587]
	GetHandleVerifier [0x00007FF6B5F1728E+4048862]
	GetHandleVerifier [0x00007FF6B5F0F173+4015811]
	GetHandleVerifier [0x00007FF6B5BE47D6+695590]
	(No symbol) [0x00007FF6B5AC0CE8]
	(No symbol) [0x00007FF6B5ABCF34]
	(No symb

In [ ]:
df = pd.read_csv('seller.csv')
df['Name_shop'] = 0
df['rank'] = 0
df['followers'] = 0
df['Positive_Feedback'] = 0
df['since'] = 0
df['Described'] = 0
df['Communication'] = 0
df['Shipping Speed'] = 0
df.head()

In [33]:
os.chdir(r'C:\Users\ACER\Downloads\uyen_crawl')
path = 'seller_image'
#Nếu như có sẵn thư mục image rồi thì bỏ qua dòng lệnh tạo thư mục
try:
    os.makedirs(path)
except:
    pass
os.chdir(path)
sellers_new = df['Id_seller']
num_seller = len(sellers_new)
for i in range(num_seller):
    url_seller_score = 'https://vi.aliexpress.com/store/feedback-score/' + str(sellers_new[i]) + '.html'
    driver.get(url_seller_score)
    time.sleep(7)
    
    

    
    try:
        img = driver.find_element(By.XPATH, '//*[@id="hd"]/div[1]/div[1]/div[1]/div/div/div[1]/div[2]/img')
        img_path = img.get_attribute('src')
        df['rank'][i] = 1
        imgs = str(sellers_new[i]) + '.jpg'
        urlretrieve(img_path, imgs)
    except:
        try:
            img = driver.find_element(By.XPATH, '//*[@id="hd"]/div[1]/div[1]/div[1]/div/div/div[1]/div[3]/img')
            img_path = img.get_attribute('src')
            df['rank'][i] = 1
            imgs = str(sellers_new[i]) + '.jpg'
            urlretrieve(img_path, imgs)
        except:
            df['rank'][i] = 0
    

    try: 
        followers = driver.find_element(By.XPATH, '//*[@id="hd"]/div[1]/div[2]/div[1]/div/div/div[2]/div[2]/span[1]').text
        df['followers'][i] = followers
    except:
        followers = np.nan
        df['followers'][i] = followers
    
    feedback = driver.find_element(By.XPATH, '//*[@id="detail-displayer"]')
    f_url = feedback.get_attribute('src')
    driver.get(f_url)
    time.sleep(5)

    name_shop = driver.find_element(By.XPATH, '//*[@id="feedback-summary"]/div[2]/table/tbody/tr[1]').text
    #print(name_shop)
    df['Name_shop'][i] = name_shop
    try:
        positive_Feedback = driver.find_element(By.XPATH, '//*[@id="feedback-summary"]/div[2]/table/tbody/tr[2]').text
        #print(Positive_Feedback)
        df['Positive_Feedback'][i] = positive_Feedback
    except: 
        df['Positive_Feedback'][i] = np.nan
    since = driver.find_element(By.XPATH, '//*[@id="feedback-summary"]/div[2]/table/tbody/tr[3]').text
    #print(since)
    df['since'][i] = since
    try:
        described = driver.find_element(By.XPATH, '//*[@id="feedback-dsr"]/div[2]/table/tbody/tr[1]/td[1]/span[2]').text
        ss = driver.find_element(By.XPATH, '//*[@id="feedback-dsr"]/div[2]/table/tbody/tr[1]/td[2]/div/em').text
        described = described + ss
        #print(described)
        communication = driver.find_element(By.XPATH, '//*[@id="feedback-dsr"]/div[2]/table/tbody/tr[2]/td[1]/span[2]').text
        ss = driver.find_element(By.XPATH, '//*[@id="feedback-dsr"]/div[2]/table/tbody/tr[2]/td[2]/div/em').text
        communication = communication + ss
        #print(communication)
        shipping_speed = driver.find_element(By.XPATH, '//*[@id="feedback-dsr"]/div[2]/table/tbody/tr[3]/td[1]/span[2]').text
        #print(shipping_speed)
        ss = driver.find_element(By.XPATH, '//*[@id="feedback-dsr"]/div[2]/table/tbody/tr[3]/td[2]/div/em').text
        shipping_speed = shipping_speed + ss

        df['Described'][i] = described
        df['Communication'][i] = communication
        df['Shipping Speed'][i] = shipping_speed
    except:
        df['Described'][i] = np.nan
        df['Communication'][i] = np.nan
        df['Shipping Speed'][i] = np.nan


    print('cao lan thu ',i+1)
os.chdir(r'C:\Users\ACER\Downloads\uyen_crawl')

In [ ]:
df.to_csv('sellers.csv', index=False, encoding='utf-8')